In [27]:
from pathlib import Path

path =  "/Bridging_the_digital_divide"

In [28]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import warnings
warnings.filterwarnings("ignore")
import os

import pandas as pd

import torch
from pkg_resources import packaging
from PIL import Image
import tqdm
from pathlib import Path
print("Torch version:", torch.__version__)


plt.style.use('ggplot')

Torch version: 1.10.2+cu113


In [6]:
data = pd.read_csv('data/results_CLIP.csv')
data.head()

,Unnamed: 0,id,country.name,country.id,region.id,type,imageRelPath,topics,place,income,CLIP score
0,24655,5fd0457ff2d9500e435933ca,Serbia,rs,eu,video,assets/5fd0457ff2d9500e435933ca/6df313d79f9c0f...,"Adding spices to food while cooking,Spices",rajkovic,1193.0,0.239534
1,16463,5fd04872f2d9500e4359465c,South Africa,za,af,video,assets/5fd04872f2d9500e4359465c/37c209cb7f9adf...,"Adding spices to food while cooking,Spices",van-rheede,448.0,0.260990
2,29818,5fd03a0bf2d9500e4358e9a7,Iran,ir,as,video,assets/5fd03a0bf2d9500e4358e9a7/de9e6e8a759f7e...,"Adding spices to food while cooking,Spices",ahmadinejad,2369.0,0.309823
3,13143,5fd025ad04167a0cab36b3a8,Brazil,br,am,video,assets/5fd025ad04167a0cab36b3a8/647b2947695589...,"Adding spices to food while cooking,Spices",cancela-reis,284.0,0.226416
4,28760,5fd02d9ca8444d0db97b4b22,Colombia,co,am,video,assets/5fd02d9ca8444d0db97b4b22/18f3b1e7cb55e1...,"Adding spices to food while cooking,Spices",cruz,1963.0,0.284779


In [7]:
path_data = 'data/'
path_processed_images = "/".join([path_data, "processed_imgs"])
path_original_images = "data/original_images/"

In [8]:
from collections import Counter
list_topics = list(data['topics'])


separate_topics = [t.lower() for topic in list_topics for t in topic.split(",") ]


print(len(separate_topics))
print(Counter(separate_topics))
set_topics = list(set(separate_topics))
print(len(set_topics))
# print(set_topics)

48055
Counter({'visit': 3366, 'toothbrush': 552, 'home': 542, 'street view': 532, 'hand washing': 530, 'family': 524, 'washing clothes/cleaning': 507, 'tv': 499, 'toilet': 494, 'hand palm': 492, 'drinking water': 488, 'kitchen sink': 480, 'tooth paste': 468, 'stove/hob': 445, 'salt': 438, 'bed': 427, 'spices': 419, 'toys': 418, 'hair brush/comb': 413, 'everyday shoes': 402, 'bedroom': 400, 'front door': 398, 'plate of food': 398, 'power outlet': 389, 'social drink': 385, 'cooking pots': 384, 'floor': 379, 'place where eating dinner': 378, 'nicest shoes': 375, 'phone': 374, 'lock on front door': 373, 'bathroom privacy': 369, 'wardrobe': 365, 'soap for hands and body': 363, 'ceiling': 362, 'hand back': 360, 'wall inside': 360, 'play area': 357, 'wall decoration': 352, 'dish washing brush/cloth': 351, 'bathroom/toilet': 350, 'refrigerator': 344, 'toilet paper': 343, 'plates': 342, 'light sources': 339, 'dish washing soap': 339, 'shampoo': 339, 'dish racks': 338, 'kitchen': 336, 'cups/mugs

In [9]:
# map each topic to list of corresponding images
dict_topic2img = {}
for list_topics, image_path in zip(data['topics'], data['imageRelPath']):
    for topic in list_topics.split(","):
        topic = topic.lower()
        if topic not in dict_topic2img:
            dict_topic2img[topic] = set()
        dict_topic2img[topic].add(image_path)

print(len(dict_topic2img))

291


In [10]:
! pip install ftfy regex tqdm
! pip install git+https://github.com/openai/CLIP.git

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-g84oyw2s
  Running command git clone -q https://github.com/openai/CLIP.git /tmp/pip-req-build-g84oyw2s
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33


In [11]:
import clip
clip.available_models()

['RN50',
 'RN101',
 'RN50x4',
 'RN50x16',
 'RN50x64',
 'ViT-B/32',
 'ViT-B/16',
 'ViT-L/14',
 'ViT-L/14@336px']

In [12]:
model, preprocess = clip.load("ViT-B/32")

model.cuda().eval()

CLIP(
  (visual): VisionTransformer(
    (conv1): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32), bias=False)
    (ln_pre): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): Sequential(
        (0): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          )
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=768, out_features=3072, bias=True)
            (gelu): QuickGELU()
            (c_proj): Linear(in_features=3072, out_features=768, bias=True)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
        (1): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          

In [13]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [18]:
original_images = [path_original_images + s for s in data['imageRelPath']]
processed_images = [path_processed_images + "/" + img_id + ".npy" for img_id in data['id']]

count = 5000
end = len(processed_images) // count + 1
print(end)
similarity_CLIP = []

list_img_features = []
for k in range(0, end):
    print(k)                    
    imgs_corrupted = []
    images_prep = []
    k_processed_images = processed_images[count*k:count*(k+1)]
    print(len(k_processed_images))
    
    for image_path in tqdm.tqdm(k_processed_images):
        if Path(image_path).is_file():         
            images_prep.append(torch.from_numpy(np.load(image_path)).to(device))
        else:
            imgs_corrupted.append(image_path)
            print(f"{image_path} corrupted")
            
    images_prep = torch.stack(images_prep)
    torch.cuda.empty_cache()
    
    with torch.no_grad():
        image_features = model.encode_image(images_prep)
        image_features /= image_features.norm(dim=-1, keepdim=True)
    list_img_features.append(image_features)
    

8
0
5000


100%|██████████| 5000/5000 [03:24<00:00, 24.48it/s]


1
5000


100%|██████████| 5000/5000 [03:19<00:00, 25.06it/s]


2
5000


...

100%|██████████| 5000/5000 [03:52<00:00, 21.53it/s]


6
5000


100%|██████████| 5000/5000 [03:48<00:00, 21.87it/s]


7
3478


100%|██████████| 3478/3478 [02:39<00:00, 21.80it/s]


In [19]:
img_features = torch.cat(list_img_features, dim=0)
img_features.shape

torch.Size([38478, 512])

In [20]:
with torch.no_grad():
    text_tokens = clip.tokenize([desc for desc in dict_topic2img]).cuda()
    text_features = model.encode_text(text_tokens).float()
    text_features /= text_features.norm(dim=-1, keepdim=True)

text_features.shape

torch.Size([291, 512])

In [21]:
similarity = text_features.cpu().numpy() @ img_features.cpu().numpy().T # dot product of image and text features
similarity.shape


(291, 38478)

In [22]:
dict_topic2score = {}
for i, topic in enumerate(dict_topic2img):
    images = dict_topic2img[topic]
    count = len(images)
#     print(topic, count)
    clip_scores = similarity[i] # clip scores between topic and images from data['imageRelPath']
        
    list_scores_img = list(zip(clip_scores, data['imageRelPath'], data['topics']))
    dict_topic2score[topic] = sorted(list_scores_img, key = lambda x: x[0], reverse=True)

#dict_topic2score

In [23]:

# load pickle module
import pickle

# create a binary pickle file 
f = open("data/dict_topic2score.pkl","wb")

# write the python object (dict) to pickle file
pickle.dump(dict_topic2score,f)

# close file
f.close()